## Setup:

1. Activate your conda environment and install library
```
pip install feast

```

2. Initialize feature repository in your location of choice and enter that folder
```
feast init my_feature_repo
cd my_feature_repo/feature_repo
```

3. Register the created feature store
```
feast apply
```

4. Spin up the ui interaction with the feature store which would be in http://127.0.0.1:8000/ after running the following command:
```
feast ui --host 127.0.0.1 --port 8000
```

Note: The UI is a beta feature and may be unpredictable. It's just an additional feature and should not affect the programmatic use of feature store

### If not already present, create a feature_store.yaml file in the feature store directory i.e (my_feature_repo/feature_repo directory) and paste the following content:

```
project: my_feature_repo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2
auth:
    type: no_auth
```

## Build a dataset

In [1]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

store = FeatureStore(repo_path="./my_feature_repo/feature_repo/") #replace this relative path with the right one if needed

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())


   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.214388  0.647115              384
1       1002 2021-04-12 08:12:10+00:00   0.017890  0.339864              122
2       1003 2021-04-12 16:40:26+00:00   0.646316  0.964426              880
3       1004 2021-04-12 15:01:12+00:00   0.793783  0.428244              689


## Load feature values into store
### By running the following command from feature repository folder using command line / terminal

```
feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")
```

You should get a successful views materialized message

If there are no problems with the feast ui (where problems are common), you will see these materialized views under Feature Views on the app ui

## Now read the features from feature store at low latency

In [2]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path="./my_feature_repo/feature_repo/") #replace this relative path with the right one if needed

feature_vector = store.get_online_features(
    features=[
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_rows=[{"driver_id": 1001}]
).to_dict()

pprint(feature_vector)

{'acc_rate': [0.8833363652229309],
 'avg_daily_trips': [281],
 'conv_rate': [0.6451407074928284],
 'driver_id': [1001]}
